In [2]:
!pip3 install torch
!pip3 install numpy
!pip3 install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 9.2 MB/s eta 0:00:0000:01m0:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 10.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

The thermal conductivity equation:
$\frac{\partial T}{\partial t} - \frac{{\partial}^2 T}{\partial x^2} = 0$

Assume $ T(t, x) = 2 + e^{-4 \pi^2 t} sin(2\pi x) + e^{-16\pi^2 t} cos(4\pi x)$, $x\in[0,1], t\in[0,5]$
$\\$Boundary condition $T_0 = T(0, x) = 2 + sin(2\pi x) + cos(4\pi x)$

In [4]:
#define amount of sample points
N = 2000

In [5]:
#in the beginning we plot graph of the thermal conductivity process
def f_real(t, x):
    return (2 + torch.exp(-4*(torch.pi**2)*t)*torch.sin(2*torch.pi*x) + torch.exp(-16*(torch.pi**2)*t)*torch.cos(4*torch.pi*x))

In [6]:
# Now we want to get test points from equation in first cell
x_data = torch.rand(N).view(-1,1)
t_data = 5 * torch.rand(N).view(-1,1)

In [7]:
#define the class PINN
class PINN(nn.Module):
    def __init__(self,input_layer=2,h1=32,h2=32,h3=32,output_layer=2):
        super().__init__()
        self.fc1 = nn.Linear(input_layer,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,h3)
        self.fc4 = nn.Linear(h3,output_layer)
        
    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = F.gelu(self.fc3(x))
        x = F.gelu(self.fc4(x))
        
        return x

In [8]:
x_phys = torch.rand(N).view(-1,1).requires_grad_(True)
t_ = 5 * torch.rand(N).view(-1,1)
t_phys = t_.requires_grad_(True)
points = torch.stack((t_phys,x_phys), -1)
points_bc = torch.stack((torch.zeros(N,1), x_phys), -1)
torch.manual_seed(41)

In [9]:
pinn = PINN()
optimizer = torch.optim.Adam(pinn.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 60, gamma=0.5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.57)

Loss1 = $\frac{1}{N} \sum_{i=1}^{N}(T(t_i, x_i)- f_{PINN}(t_i, x_i))^2$

Loss2 = $\frac{1}{N} \sum_{i=1}^{N}(T_0(x_i)- f_{PINN}(0, x_i))^2$

Loss3 = $\frac{1}{N} \sum_{i=1}^{N}(\frac{\partial}{\partial t}f_{PINN}(t_i, x_i) - \frac{{\partial}^2 }{\partial x^2}f_{PINN}(t_i, x_i))^2$

In [10]:
epochs = 500

for i in range(epochs):
    #compute MSE of T(t,x) and points that were pridicted by PINN
    network = pinn.forward(points)
    loss1 = torch.mean((f_real(t_data, x_data) - network)**2)
    y_bc = pinn.forward(points_bc)
    loss2 = torch.mean((f_real(torch.zeros_like(x_data), x_data) - y_bc)**2)
    
    #compute loss using derivatives
    dt = torch.autograd.grad(network, t_phys, torch.ones_like(network), create_graph=True)[0]
    dx = torch.autograd.grad(network, x_phys, torch.ones_like(network), create_graph=True)[0]
    dx2 = torch.autograd.grad(dx, x_phys, torch.ones_like(dx), create_graph=True)[0]
    loss3 = torch.mean((dt - dx)**2)
    
    loss = loss2 + loss3 + loss1
    loss.backward()
    
    optimizer.step()
    scheduler.step(loss)
    if i % 50 == 0:
        print(f'epoch: {i}\tamount of loss: {loss}\t')#learning rate: {scheduler.get_last_lr()}')

epoch: 0	amount of loss: 8.528813362121582	
epoch: 50	amount of loss: 1.8842103481292725	
epoch: 100	amount of loss: 1.3311703205108643	
epoch: 150	amount of loss: 1.5639346837997437	
epoch: 200	amount of loss: 1.52568519115448	
epoch: 250	amount of loss: 1.520878791809082	
epoch: 300	amount of loss: 1.520423173904419	
epoch: 350	amount of loss: 1.5203481912612915	
epoch: 400	amount of loss: 1.5202709436416626	
epoch: 450	amount of loss: 1.5201928615570068	


In [12]:
f_nn = pinn.forward(points)
print(torch.mean((f_real(t_data, x_data) - f_nn)**2))

tensor(0.2091, grad_fn=<MeanBackward0>)
